This is a performance testing with the dataset of adolescent on two model. A dual model setup is done here.

The first model Electra is going to train on the emotion and the intent of the user and the GPT-2 model is going to generate text based on the questions.

The datasets used here -> casual_lm_train.jsonl and masked_lm_train.jsonl

casual_lm_train is used for the GPT-2 training and the masked_lm train is for the Electra.

gpt-2 performance is going to evaluate by its perplexity that is how certain it's response is or other word what is it's accuracy to predict the next word.

electra's performance is evaluated by it's accuracy to the prediction of the emotion and intention of the questions.



In [ ]:
import torch
from transformers import (
    ElectraTokenizerFast, ElectraForSequenceClassification,
    GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorWithPadding
)
from datasets import Dataset, DatasetDict
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support, confusion_matrix
import pandas as pd
import numpy as np
import random
import os
import json
from sklearn.model_selection import train_test_split


torch.manual_seed(42)

In [2]:
# data from JSONL files
def load_jsonl(file_path):
    data = []
    try:
        with open(file_path, 'r') as f:
            for line in f:
                data.append(json.loads(line.strip()))
        print(f"Loaded {len(data)} samples from {file_path}")
    except FileNotFoundError:
        print(f"Error: {file_path} not found!")
        return []
    return data

masked_lm_data = load_jsonl("masked_lm_train.jsonl")
causal_lm_data = load_jsonl("causal_lm_train.jsonl")

# convert for classification format for electra
extended_data = []

# masked LM data for emotion/intent classification
for item in masked_lm_data:
    sentence1 = item.get('sentence1', '')
    sentence2 = item.get('sentence2', '')

    if not sentence1:
        continue

    # extract emotion adn intent
    text_lower = sentence1.lower()

    # predict intent based on question patterns
    if any(word in text_lower for word in ['why', 'how', 'what', 'when', 'where', '?']):
        intent = "question"
    elif any(word in text_lower for word in ['help', 'support', 'advice']):
        intent = "seeking support"
    else:
        intent = "venting"

    # predict emotion based on content patterns - IMPROVED WITH BETTER DETECTION
    if any(word in text_lower for word in ['worthless', 'nobody likes', 'hate myself', 'useless', 'sad', 'depressed', 'down', 'upset', 'lonely', 'isolated', 'alone', 'feel like crying', 'feel awful', 'terrible', 'horrible', 'empty', 'hopeless']):
        emotion = "sad"
    elif any(word in text_lower for word in ['anxious', 'anxiety', 'worry', 'worried', 'stress', 'stressed', 'nervous', 'panic', 'overwhelmed', 'can\'t stop thinking', 'overthinking', 'scared', 'afraid', 'fear']):
        emotion = "anxious"
    elif any(word in text_lower for word in ['angry', 'mad', 'frustrated', 'annoyed', 'furious', 'hate', 'pissed', 'makes me angry', 'irritated', 'rage', 'livid']):
        emotion = "angry"
    elif any(word in text_lower for word in ['mother', 'father', 'parent', 'family', 'relationship', 'independence', 'autonomy', 'confused', 'don\'t understand', 'mixed up']):
        emotion = "confused"
    elif any(word in text_lower for word in ['growth', 'development', 'maturity', 'change', 'personal', 'curious', 'wonder', 'interested']):
        emotion = "curious"
    else:
        emotion = "thoughtful"

    if sentence2:
        response = sentence2
    else:
        if emotion == "sad":
            response = "I hear that you're going through a really difficult time right now. Those feelings are valid, and you're not alone. Would you like to talk about what's been making you feel this way?"
        elif emotion == "anxious":
            response = "It sounds like you're dealing with some anxiety, which can be really challenging. These feelings are completely normal. What specifically has been making you feel anxious?"
        elif emotion == "angry":
            response = "I can hear the frustration and anger in what you're sharing. These feelings are completely understandable. Sometimes it helps to talk about what's been making you feel this way."
        elif emotion == "confused":
            response = "That's a really important question about relationships and growing up. These feelings of confusion are normal during adolescence. Let's explore this together."
        elif emotion == "curious":
            response = "That's a great question about personal growth! It shows you're thinking deeply about your development. Let's discuss what this means for you."
        else:
            response = "Thank you for sharing that with me. I appreciate your thoughtfulness. Let's discuss this further."

    extended_data.append({
        "text": sentence1,
        "emotion": emotion,
        "intent": intent,
        "response": response
    })

for item in causal_lm_data:
    text = item.get('text', '')

    if not text:
        continue

    text_lower = text.lower()

    # predict intent and emotion for causal LM data
    if any(word in text_lower for word in ['?', 'why', 'how', 'what']):
        intent = "question"
    else:
        intent = "seeking support"

    if any(word in text_lower for word in ['autonomy', 'independence', 'mother', 'father']):
        emotion = "confused"
        response = "That's an important question about family relationships and independence. Let's explore this together."
    elif any(word in text_lower for word in ['growth', 'personal', 'development']):
        emotion = "curious"
        response = "Personal growth is a key part of adolescent development. It shows you're developing self-awareness."
    elif any(word in text_lower for word in ['purpose', 'life', 'meaning']):
        emotion = "thoughtful"
        response = "Having a sense of purpose is crucial for well-being. It gives direction and meaning to your experiences."
    else:
        emotion = "thoughtful"
        response = "That's a thoughtful question. Let's discuss this in more detail."

    extended_data.append({
        "text": text,
        "emotion": emotion,
        "intent": intent,
        "response": response
    })

# training examples for better understanding and emotion detection
additional_examples = [
    {
        "text": "I feel like nobody likes me at school and I'm always alone",
        "emotion": "sad",
        "intent": "venting",
        "response": "I'm really sorry you're feeling so alone at school. That sense of isolation can be really painful. Your feelings are completely valid, and I want you to know that you matter."
    },
    {
        "text": "Why do I always feel worthless and like I can't do anything right",
        "emotion": "sad",
        "intent": "question",
        "response": "Those feelings of worthlessness are really hard to carry. It's important to know that these thoughts don't reflect your true value as a person. You're asking a brave question by reaching out."
    },
    {
        "text": "How can I deal with my anxiety it's getting really bad",
        "emotion": "anxious",
        "intent": "seeking support",
        "response": "I'm glad you're reaching out about your anxiety. That takes courage. There are definitely strategies that can help manage anxiety. Let's work through some techniques together."
    },
    {
        "text": "I'm so angry at my parents they never listen to me",
        "emotion": "angry",
        "intent": "venting",
        "response": "It's really frustrating when you feel like your parents aren't hearing you. Those angry feelings make complete sense when you don't feel understood or respected."
    },
    {
        "text": "I can't stop overthinking everything and it's making me crazy",
        "emotion": "anxious",
        "intent": "venting",
        "response": "That constant overthinking sounds exhausting. When our minds race like that, it can feel overwhelming. You're not alone in experiencing this."
    },
    {
        "text": "Why do I feel so hopeless about everything",
        "emotion": "sad",
        "intent": "question",
        "response": "Feeling hopeless can be incredibly heavy and difficult to bear. These feelings are telling me that you're really struggling right now. What's been contributing to these feelings?"
    },
    {
        "text": "I feel empty inside like nothing matters anymore",
        "emotion": "sad",
        "intent": "venting",
        "response": "That emptiness you're describing sounds really painful. When everything feels meaningless, it can be so isolating. I'm here to listen and help you work through these feelings."
    },
    {
        "text": "I hate myself for being such a failure",
        "emotion": "sad",
        "intent": "venting",
        "response": "I hear so much pain in what you're sharing. Those harsh thoughts about yourself must be exhausting to carry. You're not a failure - you're a person going through a difficult time."
    },
    {
        "text": "Everything makes me worry and I can't relax",
        "emotion": "anxious",
        "intent": "venting",
        "response": "Living with that constant worry sounds really exhausting. When anxiety affects our ability to relax, it can feel overwhelming. Let's talk about what's been triggering these worries."
    }
]

extended_data.extend(additional_examples)

labels_emotion = list(set(d['emotion'] for d in extended_data))
labels_intent = list(set(d['intent'] for d in extended_data))

print(f"Total samples: {len(extended_data)}")
print(f"Emotion labels ({len(labels_emotion)}): {labels_emotion}")
print(f"Intent labels ({len(labels_intent)}): {labels_intent}")

for d in extended_data:
    d['emotion_label'] = labels_emotion.index(d['emotion'])
    d['intent_label'] = labels_intent.index(d['intent'])

# train-test split and dataset conversion
train_data, test_data = train_test_split(extended_data, test_size=0.15, random_state=42)

train_dataset = Dataset.from_pandas(pd.DataFrame(train_data))
test_dataset = Dataset.from_pandas(pd.DataFrame(test_data))

print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")

print("\nSample training data:")
for i in range(min(3, len(train_data))):
    sample = train_data[i]
    print(f"Text: {sample['text'][:80]}...")
    print(f"Emotion: {sample['emotion']}, Intent: {sample['intent']}")
    print(f"Response: {sample['response'][:80]}...")
    print("-" * 40)

Loaded 600 samples from masked_lm_train.jsonl
Loaded 600 samples from causal_lm_train.jsonl
Total samples: 1209
Emotion labels (6): ['confused', 'thoughtful', 'curious', 'anxious', 'sad', 'angry']
Intent labels (3): ['question', 'seeking support', 'venting']
Training samples: 1027
Test samples: 182

Sample training data:
Text: How does your relationship with your mother affect your autonomy?...
Emotion: confused, Intent: question
Response: That's an important question about family relationships and independence. Let's ...
----------------------------------------
Text: Why is purpose in life important for teenagers?...
Emotion: thoughtful, Intent: question
Response: Having a sense of purpose is crucial for well-being. It gives direction and mean...
----------------------------------------
Text: What does a high personal growth score mean during adolescence?...
Emotion: curious, Intent: question
Response: Personal growth is a key part of adolescent development. It shows you're develop...

In [3]:
# electra classifier for emotion and intent prediction

classifier_model = ElectraForSequenceClassification.from_pretrained(
    "google/electra-small-discriminator",
    num_labels=len(labels_emotion)
)
classifier_tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-small-discriminator")

def tokenize_and_encode(batch):
    return classifier_tokenizer(batch['text'], truncation=True, padding=True, max_length=128)

# tokenization of datasets
train_tokenized = train_dataset.map(tokenize_and_encode, batched=True)
test_tokenized = test_dataset.map(tokenize_and_encode, batched=True)

train_tokenized = train_tokenized.rename_column("emotion_label", "labels")
test_tokenized = test_tokenized.rename_column("emotion_label", "labels")

train_tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# training arguments
classifier_args = TrainingArguments(
    output_dir="./classifier_output",
    logging_steps=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    seed=42,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
)

# trainer
trainer = Trainer(
    model=classifier_model,
    args=classifier_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=classifier_tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=classifier_tokenizer),
    compute_metrics=lambda p: {
        "accuracy": accuracy_score(
            p.label_ids, np.argmax(p.predictions, axis=1)
        )
    }
)

print("Training ELECTRA classifier...")
trainer.train()
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

classifier_model.save_pretrained("./electra_emotion_classifier")
classifier_tokenizer.save_pretrained("./electra_emotion_classifier")

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1027 [00:00<?, ? examples/s]

Map:   0%|          | 0/182 [00:00<?, ? examples/s]

/tmp/ipykernel_7886/3845594068.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Training ELECTRA classifier...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.118700,0.060861,0.989011
2,0.007400,0.022751,0.989011
3,0.003900,0.018097,0.989011
4,0.003000,0.016690,0.994505
5,0.002500,0.016263,0.994505


Evaluation results: {'eval_loss': 0.016262898221611977, 'eval_accuracy': 0.9945054945054945, 'eval_runtime': 0.3841, 'eval_samples_per_second': 473.778, 'eval_steps_per_second': 119.746, 'epoch': 5.0}


('./electra_emotion_classifier/tokenizer_config.json',
 './electra_emotion_classifier/special_tokens_map.json',
 './electra_emotion_classifier/vocab.txt',
 './electra_emotion_classifier/added_tokens.json',
 './electra_emotion_classifier/tokenizer.json')

In [4]:
# training for gpt2
print("Preparing GPT-2 training data...")

prompted_samples = []

for d in extended_data:
    prompted_samples.append({
        "input": f"emotion: {d['emotion']}, intent: {d['intent']}\nUser: {d['text']}\nAssistant:",
        "output": d['response']
    })

print(f"Total GPT-2 training samples: {len(prompted_samples)}")

# model loading
gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_tokenizer.pad_token = gpt_tokenizer.eos_token
gpt_model = GPT2LMHeadModel.from_pretrained("gpt2")

def encode_gpt(example):
    prompt = example['input']
    response = example['output']

    # Create a more focused training format
    full_text = f"{prompt} {response}<|endoftext|>"

    encoding = gpt_tokenizer(
        full_text,
        truncation=True,
        padding="max_length",
        max_length=256,
        return_tensors="pt"
    )

    # labels for causal language modeling
    labels = encoding["input_ids"].clone()

    # Find where the assistant response starts to focus training
    prompt_tokens = gpt_tokenizer(prompt, add_special_tokens=False)["input_ids"]
    prompt_length = len(prompt_tokens)

    # Mask the prompt part so model focuses on learning responses
    if prompt_length < labels.shape[1]:
        labels[0, :prompt_length] = -100

    encoding["labels"] = labels.squeeze(0)
    return {k: v.squeeze(0) for k, v in encoding.items()}

# dataset to train and split
gpt_train_data, gpt_test_data = train_test_split(prompted_samples, test_size=0.1, random_state=42)

gpt_train_dataset = Dataset.from_list(gpt_train_data)
gpt_test_dataset = Dataset.from_list(gpt_test_data)

print(f"GPT-2 training samples: {len(gpt_train_dataset)}")
print(f"GPT-2 test samples: {len(gpt_test_dataset)}")

gpt_train_dataset = gpt_train_dataset.map(encode_gpt)
gpt_test_dataset = gpt_test_dataset.map(encode_gpt)

gpt_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
gpt_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# training arguments for GPT-2
gpt_args = TrainingArguments(
    output_dir="./gpt2_response_generator",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
    gradient_accumulation_steps=4,
    warmup_steps=100,
    learning_rate=3e-5,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
)

# trainer
gpt_trainer = Trainer(
    model=gpt_model,
    args=gpt_args,
    train_dataset=gpt_train_dataset,
    eval_dataset=gpt_test_dataset,
    tokenizer=gpt_tokenizer,
    data_collator=DataCollatorWithPadding(gpt_tokenizer, padding=True)
)

print("Training GPT-2 response generator...")
gpt_trainer.train()
gpt_eval_results = gpt_trainer.evaluate()
print(f"GPT-2 evaluation results: {gpt_eval_results}")

# Calculate perplexity for better evaluation of generative model
import math
eval_loss = gpt_eval_results['eval_loss']
perplexity = math.exp(eval_loss)
print(f"GPT-2 Perplexity: {perplexity:.2f}")
print(f"GPT-2 Evaluation Loss: {eval_loss:.4f}")

# Save the model
gpt_model.save_pretrained("./gpt2_adolescent_chatbot_final")
gpt_tokenizer.save_pretrained("./gpt2_adolescent_chatbot_final")

Preparing GPT-2 training data...
Total GPT-2 training samples: 1209
GPT-2 training samples: 1088
GPT-2 test samples: 121
GPT-2 training samples: 1088
GPT-2 test samples: 121


Map:   0%|          | 0/1088 [00:00<?, ? examples/s]

Map:   0%|          | 0/121 [00:00<?, ? examples/s]

/tmp/ipykernel_7886/1470145640.py:82: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  gpt_trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Training GPT-2 response generator...


Epoch,Training Loss,Validation Loss
1,0.014300,0.025930
2,0.022800,0.025999
3,0.011600,0.025878
4,0.015400,0.024932
5,0.012000,0.025138


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


GPT-2 evaluation results: {'eval_loss': 0.024932222440838814, 'eval_runtime': 1.3623, 'eval_samples_per_second': 88.823, 'eval_steps_per_second': 44.778, 'epoch': 5.0}
GPT-2 Perplexity: 1.03
GPT-2 Evaluation Loss: 0.0249


('./gpt2_adolescent_chatbot_final/tokenizer_config.json',
 './gpt2_adolescent_chatbot_final/special_tokens_map.json',
 './gpt2_adolescent_chatbot_final/vocab.json',
 './gpt2_adolescent_chatbot_final/merges.txt',
 './gpt2_adolescent_chatbot_final/added_tokens.json')

In [5]:
# Inference Functions for Two-Model Chatbot Architecture

def classify_emotion_intent(text):

    # Electra model loading to the device
    device = next(classifier_model.parameters()).device

    inputs = classifier_tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    inputs = {k: v.to(device) for k, v in inputs.items()}

    classifier_model.eval()
    with torch.no_grad():
        outputs = classifier_model(**inputs)
        pred = torch.argmax(outputs.logits, dim=1).item()
        confidence = torch.softmax(outputs.logits, dim=1).max().item()

    return {
        "emotion": labels_emotion[pred],
        "confidence": confidence,
        "all_scores": {labels_emotion[i]: score.item() for i, score in enumerate(torch.softmax(outputs.logits, dim=1)[0])}
    }

def generate_response(text, max_length=80, temperature=0.8):
    # gpt2 model loading to the device
    classification = classify_emotion_intent(text)
    emotion = classification["emotion"]

    # Determine intent from the text for better prompting
    text_lower = text.lower()
    if any(word in text_lower for word in ['why', 'how', 'what', 'when', 'where']):
        intent = "question"
    elif any(word in text_lower for word in ['help', 'deal with', 'handle', 'support']):
        intent = "seeking support"
    else:
        intent = "venting"

    # Create a more specific prompt for better responses
    prompt = f"emotion: {emotion}, intent: {intent}\nUser: {text}\nAssistant:"

    device = next(gpt_model.parameters()).device

    # generate response with better parameters
    input_ids = gpt_tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    gpt_model.eval()

    with torch.no_grad():
        output = gpt_model.generate(
            input_ids,
            max_new_tokens=max_length,
            temperature=temperature,
            do_sample=True,
            top_p=0.9,  # Nucleus sampling for better quality
            top_k=50,   # Top-k sampling
            pad_token_id=gpt_tokenizer.pad_token_id,
            eos_token_id=gpt_tokenizer.eos_token_id,
            repetition_penalty=1.2,  # Reduce repetition
            no_repeat_ngram_size=3   # Avoid repeating 3-grams
        )

    full_response = gpt_tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract only the assistant's response
    if "Assistant:" in full_response:
        assistant_response = full_response.split("Assistant:")[-1].strip()
    else:
        assistant_response = "I understand you're going through something difficult. Can you tell me more about how you're feeling?"

    # Clean up the response
    assistant_response = assistant_response.replace("<|endoftext|>", "").strip()

    # If response is too short or generic, provide a fallback
    if len(assistant_response) < 10 or assistant_response in ["", " "]:
        if emotion == "sad":
            assistant_response = "I can hear that you're really struggling right now. Those feelings are completely valid. What's been weighing on your mind?"
        elif emotion == "anxious":
            assistant_response = "It sounds like you're dealing with some anxiety. That can be really overwhelming. Would you like to talk about what's been making you feel this way?"
        elif emotion == "angry":
            assistant_response = "I can sense your frustration. Those feelings are completely understandable. What's been making you feel this way?"
        else:
            assistant_response = "Thank you for sharing that with me. I'd like to understand better - can you tell me more about what you're experiencing?"

    return {
        "response": assistant_response,
        "emotion_detected": emotion,
        "confidence": classification["confidence"],
        "emotion_scores": classification["all_scores"]
    }

# Testing
print("Testing Two-Model Chatbot Architecture")
print("=" * 50)

test_inputs = [
    "I feel like nobody likes me at school.",
    "Why do I always feel worthless?",
    "How can I deal with my anxiety?",
    "I'm angry at my parents for not understanding me.",
    "What does personal growth mean for teenagers?",
    "How does my relationship with my mother affect my independence?"
]

for i, user_input in enumerate(test_inputs, 1):
    print(f"\n Test {i}: {user_input}")
    print("-" * 30)

    result = generate_response(user_input)

    print(f"Emotion Detected: {result['emotion_detected']} (confidence: {result['confidence']:.2f})")
    print(f"Assistant Response: {result['response']}")

    print(f"\nEmotion Analysis:")
    for emotion, score in result['emotion_scores'].items():
        print(f"   {emotion}: {score:.3f}")

print("\n" + "=" * 50)
print("Model Evaluation on Test Set")
print("=" * 50)

correct_predictions = 0
total_predictions = 0

for item in test_data[:10]:
    prediction = classify_emotion_intent(item['text'])
    actual_emotion = item['emotion']
    predicted_emotion = prediction['emotion']

    if predicted_emotion == actual_emotion:
        correct_predictions += 1
    total_predictions += 1

    print(f"Text: {item['text'][:50]}...")
    print(f"Actual: {actual_emotion} | Predicted: {predicted_emotion} | {'Correct' if predicted_emotion == actual_emotion else 'Wrong'}")
    print()

accuracy = correct_predictions / total_predictions
print(f"Classification Accuracy: {accuracy:.2f} ({correct_predictions}/{total_predictions})")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Testing Two-Model Chatbot Architecture

 Test 1: I feel like nobody likes me at school.
------------------------------
Emotion Detected: sad (confidence: 0.27)
Assistant Response: That's true! In fact, the most common response you hear is "I'm not really that into academics." This may be a result of an individual feeling isolated or ignored by their peers.

Emotion Analysis:
   confused: 0.083
   thoughtful: 0.090
   curious: 0.067
   anxious: 0.251
   sad: 0.272
   angry: 0.237

 Test 2: Why do I always feel worthless?
------------------------------
Emotion Detected: sad (confidence: 0.27)
Assistant Response: That's true! In fact, the most common response you hear is "I'm not really that into academics." This may be a result of an individual feeling isolated or ignored by their peers.

Emotion Analysis:
   confused: 0.083
   thoughtful: 0.090
   curious: 0.067
   anxious: 0.251
   sad: 0.272
   angry: 0.237

 Test 2: Why do I always feel worthless?
------------------------------
Emoti

In [6]:
# Interactive Testing Cell - Test Your Own Questions
print("Test the chatbot with your own questions on different topics!")
print("Type 'quit' to exit")
print("-" * 50)

def interactive_chat():
    while True:
        user_input = input("\nYour question: ")

        if user_input.lower() in ['quit', 'exit', 'stop']:
            print("Thanks for testing! Goodbye!")
            break

        if not user_input.strip():
            print("Please enter a question.")
            continue

        print(f"\nProcessing: '{user_input}'")
        print("-" * 40)

        try:
            # Get the response from our chatbot
            result = generate_response(user_input)

            print(f"Emotion Detected: {result['emotion_detected']} (confidence: {result['confidence']:.2f})")
            print(f"Assistant Response: {result['response']}")

            print(f"\nDetailed Emotion Analysis:")
            sorted_emotions = sorted(result['emotion_scores'].items(), key=lambda x: x[1], reverse=True)
            for emotion, score in sorted_emotions:
                bar = "||" * int(score * 20)  # Visual bar
                print(f"   {emotion:>10}: {score:.3f} {bar}")

        except Exception as e:
            print(f"Error occurred: {str(e)}")
            print("Please try a different question.")

# Test with some diverse questions first
test_questions = [
    "I'm having trouble sleeping because I keep thinking about my future",
    "My friends seem to be moving on without me and I feel left behind",
    "I don't know what I want to do with my life and it scares me",
    "I feel pressure to be perfect all the time",
    "I'm struggling with my identity and who I really am",
    "I feel like I'm not good enough compared to others on social media",
    "I'm worried about climate change and the future of the world",
    "I feel like my teachers don't understand me",
    "I'm having conflicts with my siblings",
    "I feel overwhelmed by all the expectations people have of me"
]

print("\nTesting with diverse questions first:")
print("=" * 50)

for i, question in enumerate(test_questions, 1):
    print(f"\nTest {i}: {question}")
    print("-" * 30)

    try:
        result = generate_response(question)
        print(f"Emotion: {result['emotion_detected']} ({result['confidence']:.2f})")
        print(f"Response: {result['response']}")

        # Check if response is too generic/repetitive
        generic_phrases = [
            "That's true! In fact",
            "the most common response",
            "not really that into academics"
        ]

        is_generic = any(phrase in result['response'] for phrase in generic_phrases)
        if is_generic:
            print("WARNING: Response appears to be generic/repetitive")

    except Exception as e:
        print(f"Error: {str(e)}")

print(f"\n{'='*50}")
print("Now you can test with your own questions!")
print("Try asking about topics like:")
print("- Relationships and friendships")
print("- School and academic pressure")
print("- Future career concerns")
print("- Social media and self-image")
print("- Family dynamics")
print("- Personal growth and identity")
print("- Mental health and coping")
print(f"{'='*50}")

interactive_chat()

Test the chatbot with your own questions on different topics!
Type 'quit' to exit
--------------------------------------------------

Testing with diverse questions first:

Test 1: I'm having trouble sleeping because I keep thinking about my future
------------------------------
Emotion: sad (0.27)
Response: That's not a good sign. It shows you're struggling with your thoughts.

Test 2: My friends seem to be moving on without me and I feel left behind
------------------------------
Emotion: sad (0.27)
Response: That's not a good sign. It shows you're struggling with your thoughts.

Test 2: My friends seem to be moving on without me and I feel left behind
------------------------------
Emotion: sad (0.27)
Response: That's not true. Those of us who're together can make a difference in the world.

Test 3: I don't know what I want to do with my life and it scares me
------------------------------
Emotion: sad (0.27)
Response: That's not true. Those of us who're together can make a differen

Electra performance calculation

In [8]:
def evaluate_electra_performance():
    print("Electra emotion and intent classifier performance")
    print("="*50)

    prediction = []
    true_labels = []
    confidence = []

    classifier_model.eval()
    for item in test_data:
        result = classify_emotion_intent(item['text'])
        prediction.append(result['emotion'])
        true_labels.append(item['emotion'])
        confidence.append(result['confidence'])

    # convertion to label index for metrics
    pred_indices = [labels_emotion.index(pred) for pred in prediction]
    true_indices = [labels_emotion.index(true) for true in true_labels]

    # metrics calculation
    accuracy = accuracy_score(true_indices, pred_indices)
    precision, recall, fl, support = precision_recall_fscore_support(true_indices, pred_indices, average='weighted')

    print(f"Overall accuracy: {accuracy:.4f}")
    print(f"Weighted precision: {precision:.4f}")
    print(f"Weighted Recall: {recall:.4f}")
    print(f"Weighted Fl_Score: {fl:.4f}")
    print(f"Average Confidence: {np.mean(confidence):.4f}")

    print("/nPer-class Performance:")
    print("="*50)

    class_report = classification_report(
        true_indices, pred_indices,
        target_names=labels_emotion,
        output_dict=True,
    )

    for emotion in labels_emotion:
        metrics = class_report[emotion]
        print(f"{emotion:>10}: P={metrics['precision']:.3f}, R={metrics['recall']:.3f}, F1={metrics['f1-score']:.3f}")

    # confusion metrics
    cm = confusion_matrix(true_indices, pred_indices)
    print(f"\nConfusion matrix:")
    print("Actual\\Predicted", end="")
    for emotion in labels_emotion:
        print(f"{emotion:>8}", end="")
    print()

    for i, emotion in enumerate(labels_emotion):
        print(f"{emotion:>12}", end="")
        for j in range(len(labels_emotion)):
            print(f"{cm[i, j]:>8}", end="")
        print()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'avg_confidence': np.mean(confidence),
        'confusion_matrix': cm,
        'class_report': class_report
    }

GPT-2 performance calculation

In [ ]:
def evaluate_gpt2_performance():
    print("GPT-2 Performance")
    print("="*50)

    if 'gpt_eval_results' in globals():
        eval_loss = gpt_eval_results['eval_loss']
        perplexity = math.exp(eval_loss)
        print(f"Perplexity: {perplexity:.2f}")
        print(f"Evaluation loss: {eval_loss:.4f}")

    # response quality metrics
    response_length = []
    response_diversity = set()
    appropriate_response = 0
    total_response = 0

    test_sample = test_data[:20]

    print("\nResponse Quality Analysis:")
    print("=" * 30)

    for item in test_sample:
        result = generate_response(item['text'], max_length=60)
        response =  result['response']

        # response length
        response_length.append(len(response.split()))
        response_diversity.add(response)

        # basic heuristics
        is_appropriate = (
            len(response.split()) >= 5 and
            len(response.split()) <= 100 and
            not any(phrase in response.lower() for phrase in ['i don\'t know', 'sorry, i can\'t'])and
            any(word in response.lower() for word in ['feel', 'understand', 'help', 'support', 'you'])
        )

        if is_appropriate:
            appropriate_responses += 1
        total_responses += 1

        print(f"Input: {item['text'][:50]}...")
        print(f"Response: {response[:80]}...")
        print(f"Length: {len(response.split())} words | Appropriate: {'Yes' if is_appropriate else 'No'}")
        print()

    # Calculate metrics
    avg_length = np.mean(response_length)
    diversity_ratio = len(response_diversity) / len(test_sample)
    appropriateness_rate = appropriate_responses / total_responses

    print(f"Average Response Length: {avg_length:.1f} words")
    print(f"Response Diversity Ratio: {diversity_ratio:.3f}")
    print(f"Appropriateness Rate: {appropriateness_rate:.3f}")

    return {
        'perplexity': perplexity if 'gpt_eval_results' in globals() else None,
        'avg_length': avg_length,
        'diversity_ratio': diversity_ratio,
        'appropriateness_rate': appropriateness_rate,
        'response_lengths': response_length
    }